In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
from imblearn.over_sampling import SMOTE

In [ ]:
cr_data = pd.read_csv("credit_risk_dataset.csv")

In [ ]:
risk_level_mapping = {'Low': 0, 'Medium': 1, 'High': 2}

In [ ]:
cr_data['risk_level'] = cr_data['risk_level'].map(risk_level_mapping)

In [ ]:
categorical_columns = cr_data.select_dtypes(include=['object']).columns.tolist()

In [ ]:
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(cr_data[categorical_columns]).astype(int)

In [ ]:
joblib.dump(encoder, "encoder.pkl")

In [ ]:
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

In [ ]:
# We delete the one hot coded and binary columns so that we can remain with data to scale
data_to_scale = cr_data.drop(['marital_status', 'education_level', 'relationship_to_student', 'employment_status', 'existing_loans', 
    'financial_counseling', 'risk_level', 'risk_score'], axis=1)

In [ ]:
scaler = StandardScaler()

In [ ]:
# We scale the data 
scaled_data = scaler.fit_transform(data_to_scale)

In [ ]:
joblib.dump(scaler, "scaler.pkl")

In [ ]:
scaled_df = pd.DataFrame(scaled_data, columns=['age', 'no_of_dependants', 'income_in_kes', 'additional_income', 'employment_length', 
    'guarantor_credit_score', 'outstanding_loan_amount', 'monthly_repayment_amount', 'monthly_expenses', 'missed_payments_last_year'])

In [ ]:
# Convert boolean columns (True/False) to numerical (1/0)
boolean_columns = ['existing_loans', 'financial_counseling']
boolean_columns = cr_data[boolean_columns].astype(int)

In [ ]:
cr_data_combined = pd.concat([scaled_df, one_hot_df, boolean_columns],axis=1)
cr_data_combined['risk_level'] = cr_data['risk_level']

In [ ]:
target = cr_data_combined['risk_level']
features = cr_data_combined.drop('risk_level', axis=1)

In [ ]:
smote = SMOTE()

In [ ]:
balanced_features, balanced_target = smote.fit_resample(features,target)

In [ ]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(balanced_features, balanced_target, test_size=0.2, random_state=42)

## Logistic Regression Classifier

In [ ]:
# Initialize Logistic Regression model
logit_model = LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
logit_model.fit(X_train, y_train)

In [ ]:
joblib.dump(logit_model, "lr_risk_model.pkl")

In [ ]:
# Evaluate
lr_pred = logit_model.predict(X_test)

In [ ]:
# Calculate performance metrics
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Model Evaluation ===")
    print("Logistic Regression - Accuracy:", accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("\n")

In [ ]:
# Evaluate the model
evaluate_model(y_test, lr_pred, "Credit Risk Prediction")

## Random Forest Classifier

In [ ]:
# Train
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
joblib.dump(rf_model, "rf_risk_model.pkl")

In [ ]:
# Evaluate
rf_pred = rf_model.predict(X_test)

In [ ]:
# Evaluate the model
evaluate_model(y_test, rf_pred, "Credit Risk Prediction")

In [ ]:
features_imp_rf = pd.DataFrame({'features': balanced_features.columns, 'rf_imp': rf_model.feature_importances_})

## K-Nearest Neighbors

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn_model.fit(X_train, y_train)

In [ ]:
knn_pred = knn_model.predict(X_test)

In [ ]:
evaluate_model(y_test, knn_pred, "Credit Risk Prediction")

## XGBoost Classifier

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)

In [ ]:
xgb_model.fit(X_train, y_train)

In [ ]:
xgb_pred = xgb_model.predict(X_test)

In [ ]:
evaluate_model(y_test, xgb_pred, "Credit Risk Prediction")

## Support Vector Machine

In [ ]:
svm_model = SVC(kernel='rbf', C=1, gamma='scale')

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
svm_pred = svm_model.predict(X_test)

In [ ]:
evaluate_model(y_test, svm_pred, "Credit Risk Prediction")

## Decision Tree Classifier

In [ ]:
dtc_model = DecisionTreeClassifier(max_depth=5, random_state=42)

In [ ]:
dtc_model.fit(X_train, y_train)

In [ ]:
dtc_pred = dtc_model.predict(X_test)

In [ ]:
evaluate_model(y_test, dtc_pred, "Credit Risk Prediction")